MultiRAT preprocessing
================
Joanes Grandjean

![rat art](../assets/img/rat_art.png)

# Purpose
In this section I generate the bare bone BIDS folder metadata ahead of an upload to opennenuro.org. Datasets acquired with standardRAT protocol were generated using proper BIDS converter (bidscoin and brkraw), so they contain the right metadata. Other datasets were supplied `as is` by contributing parties. See   [asset preparation](scripts/proj_asset.ipynb) for the detail on how supplied data were converted to NIFTI and BIDS.   

If re-using some of the scripts, please follow citations guidelines for
the software used. I’ve provided the links to the software wherever
possible. See also the [license](../LICENSE.md) for this software.


In [3]:
# init variables
init_folder='/home/traaffneu/joagra/code/MultiRat'  # location of the codes
analysis_folder='/project/4180000.19/multiRat'      # location of the bids folder and results
df_path='../assets/table/standardRat_20220408.tsv'    # meta-data table


In [4]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(df_path, sep='\t')  # load the table

BIDs 1.6.0 requires the following for fMRI. 
1. RepetitionTime
2. TaskName (for func)

The rest is recommended or optional. I am however unsure of the precision of all parameters provided by the users for their respective datasets. For this reason, I only added the following meta-data items to the .json files. 

MagneticFieldStrength
PulseSequenceType
EchoTime

The full table with raw user inputs is provided in the ./assets/table folder. 

In [57]:
import os
import json

for index, row in df.iterrows():

    subject_ID=str(row['SubjID'])

    jsonfilename=os.path.join(analysis_folder, 'bids_standard',subject_ID,'func',subject_ID+'_task-rest_bold.json')


    with open(jsonfilename) as f:
        data = json.load(f)

    key_list = []

    #remove anything that could identify the center
    data.pop('AcquisitionDuration', None)
    data.pop('Project', None)
    data.pop('MRinvestigator', None)
    data.pop('laboratory', None)
    data.pop('Strain', None)
    data.pop('InstitutionName', None)
    data.pop('InstitutionAddress', None)
    data.pop('InstitutionalDepartmentName', None)
    data.pop('StationName', None)
    data.pop('ProcedureStepDescription', None)
    data.pop('DeviceSerialNumber', None)
    data.pop('MRTransmitCoilSequence', None)
    data.pop('PhaseEncodingDirection', None)
    data.pop('TaskName', None)

    

    data['RepetitionTime']=1

    #remove one of two exclusive values 
    if "VolumeTiming" in data:
        if "RepetitionTime" in data:
            data.pop('RepetitionTime', None) 

    #remove empty values
    for key, value in data.items():
        if value == 'Value was not specified': 
            key_list.append(key)


    for key in key_list:
        data.pop(key, None)


    #save json
    with open(jsonfilename, 'w') as f:
        json.dump(data, f)
        

In [55]:
# correct one dataset's timing info in the header

import os
import nibabel as nib

df_sub = df.loc[(df['DatasetID'] == 3008)]

for index, row in df_sub.iterrows():


    subject_ID=str(row['SubjID'])


    funcfilename=os.path.join(analysis_folder, 'bids_standard',subject_ID,'func',subject_ID+'_task-rest_bold.nii.gz')
    print(funcfilename)
    n1_img = nib.load(funcfilename)
    print(n1_img.header)
    n1_img.header['xyzt_units'] = 10


    n1_img.to_filename(funcfilename)

/project/4180000.19/multiRat/bids_standard/sub-300800/func/sub-300800_task-rest_bold.nii.gz
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [   4   64   18   64 1000    1    1    1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1.  0.4 1.  0.4 1.  1.  1.  1. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 8
glmin           : 0
descrip         : b'Created with ExploreDTI'
aux_file        : b'none'
qform_code      : unknown
sform_code      : scanner
quatern_b       : 0.0
quatern_c       : 0.0
quatern

In [56]:
# ! bids validator in python is moody. seemly working using website (minus some warnings)
from bids_validator import BIDSValidator
BIDSValidator().is_bids(os.path.join(analysis_folder,'bids_standard'))


False